<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Desafío 2 - Custom embedddings con Gensim



### Objetivo
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizo como dataset los dos libros de Don Quijote de la Mancha de Miguel de Cervantes Saavedra.

In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('quijote.txt', sep='/n', header=None, engine='python')
df.head()

,0
0,El ingenioso hidalgo don Quijote de la Mancha
1,TASA
2,"Yo, Juan Gallo de Andrada, escribano de Cámara..."
3,"los que residen en su Consejo, certifico y doy..."
4,los señores dél un libro intitulado El ingenio...


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 31619


### 1 - Preprocesamiento

Hay que agregar a los filtros de `text_to_word_sequence` algunos símbolos extra que se utilizan en el idioma español (¡¿«»)

In [5]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0], filters='«»¡!"#$%&()*+,-./:;<=>¿?@[\\]^_`{|}~\t\n—'))

In [6]:
# Demos un vistazo
sentence_tokens[:3]

[['el', 'ingenioso', 'hidalgo', 'don', 'quijote', 'de', 'la', 'mancha'],
 ['tasa'],
 ['yo',
  'juan',
  'gallo',
  'de',
  'andrada',
  'escribano',
  'de',
  'cámara',
  'del',
  'rey',
  'nuestro',
  'señor',
  'de']]

### 2 - Crear los vectores (word2vec)

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

#### Skip-Gram 1 (window=2)

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
sg_2 = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                window=2,       # cant de palabras antes y desp de la predicha
                vector_size=300,       # dimensionalidad de los vectores
                negative=20,    # cantidad de negative samples... 0 es no se usa
                workers=1,      # si tienen más cores pueden cambiar este valor
                sg=1)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
sg_2.build_vocab(sentence_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", sg_2.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(sg_2.wv.index_to_key))

Cantidad de docs en el corpus: 31619
Cantidad de words distintas en el corpus: 5299


#### Skip-Gram 2 (window=5)

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
sg_5 = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                window=5,       # cant de palabras antes y desp de la predicha
                vector_size=300,       # dimensionalidad de los vectores
                negative=20,    # cantidad de negative samples... 0 es no se usa
                workers=1,      # si tienen más cores pueden cambiar este valor
                sg=1)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
sg_5.build_vocab(sentence_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", sg_5.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(sg_5.wv.index_to_key))

Cantidad de docs en el corpus: 31619
Cantidad de words distintas en el corpus: 5299


#### CBOW 1 (window=2)



In [10]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
cbow_2 = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                  window=2,       # cant de palabras antes y desp de la predicha
                  vector_size=300,       # dimensionalidad de los vectores
                  negative=20,    # cantidad de negative samples... 0 es no se usa
                  workers=1,      # si tienen más cores pueden cambiar este valor
                  sg=0)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
cbow_2.build_vocab(sentence_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", cbow_2.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(cbow_2.wv.index_to_key))

Cantidad de docs en el corpus: 31619
Cantidad de words distintas en el corpus: 5299


#### CBOW 2 (window=5)

In [11]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
cbow_5 = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                  window=5,       # cant de palabras antes y desp de la predicha
                  vector_size=300,       # dimensionalidad de los vectores
                  negative=20,    # cantidad de negative samples... 0 es no se usa
                  workers=1,      # si tienen más cores pueden cambiar este valor
                  sg=0)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
cbow_5.build_vocab(sentence_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", cbow_5.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(cbow_5.wv.index_to_key))

Cantidad de docs en el corpus: 31619
Cantidad de words distintas en el corpus: 5299


### 3 - Entrenar embeddings

In [12]:
# Entrenamos los modelos generadores de vectores
# Utilizamos nuestro callback

models = {
    "Skip-Gram con ventana 2": sg_2,
    "Skip-Gram con ventana 5": sg_5,
    "CBOW con ventana 2": cbow_2,
    "CBOW con ventana 5": cbow_5
}

for name, model in models.items():
  print(f"Entrenando {name}\n")
  model.train(sentence_tokens,
              total_examples=model.corpus_count,
              epochs=50,
              compute_loss = True,
              callbacks=[callback()]
              )
  print()

Entrenando Skip-Gram con ventana 2

Loss after epoch 0: 2509553.0
Loss after epoch 1: 1772175.0
Loss after epoch 2: 1653907.5
Loss after epoch 3: 1633828.5
Loss after epoch 4: 1589399.0
Loss after epoch 5: 1542882.0
Loss after epoch 6: 1522711.0
Loss after epoch 7: 1507586.0
Loss after epoch 8: 1485059.0
Loss after epoch 9: 1472954.0
Loss after epoch 10: 1410393.0
Loss after epoch 11: 1389024.0
Loss after epoch 12: 1373740.0
Loss after epoch 13: 1362694.0
Loss after epoch 14: 1346404.0
Loss after epoch 15: 1334560.0
Loss after epoch 16: 1324560.0
Loss after epoch 17: 1305664.0
Loss after epoch 18: 1296300.0
Loss after epoch 19: 1282840.0
Loss after epoch 20: 1271840.0
Loss after epoch 21: 1259820.0
Loss after epoch 22: 1244202.0
Loss after epoch 23: 1205632.0
Loss after epoch 24: 1193092.0
Loss after epoch 25: 1180240.0
Loss after epoch 26: 1170280.0
Loss after epoch 27: 1161656.0
Loss after epoch 28: 1153872.0
Loss after epoch 29: 1150516.0
Loss after epoch 30: 1141052.0
Loss after ep

### 4 - Ensayar

#### Skip-Gram 1

In [13]:
# Palabras que MÁS se relacionan con...:
sg_2.wv.most_similar(positive=["quijote"], topn=10)

[('don', 0.6591389775276184),
 ('jerónimo', 0.442010760307312),
 ('apaleado', 0.3935890197753906),
 ('gaspar', 0.37267908453941345),
 ('tarfe', 0.3676985800266266),
 ('sentar', 0.367477685213089),
 ('belianís', 0.3658357560634613),
 ('ésas', 0.3628139793872833),
 ('hiciéronlo', 0.3620290160179138),
 ('malparado', 0.3610021471977234)]

Se puede observar que con Skip-Gram con ventana 2, la palabra más similar a "quijote" es "don" ya que el personaje principal es "Don Quijote". Además se pueden observar múltiples personajes secundarios o personajes de literatura conocidos por Don Quijote junto con palabras muy utilizadas por este personaje.

In [14]:
# Palabras que MÁS se relacionan con...:
sg_2.wv.most_similar(positive=["sancho"], topn=10)

[('panza', 0.4396946132183075),
 ('dígote', 0.4296571612358093),
 ('majadero', 0.42393758893013),
 ('ta', 0.39974963665008545),
 ('estemos', 0.394745409488678),
 ('decid', 0.39415425062179565),
 ('desesperaba', 0.3669798970222473),
 ('oyéndole', 0.3660237491130829),
 ('sentar', 0.3643604815006256),
 ('jerónimo', 0.35951119661331177)]

Para la palabra "sancho" obviamente la palabra más similar en este contexto es "panza" por la misma razón que con "don quijote". También se observan palabras que comunmente menciona el personaje y emociones que el personaje posee.

In [17]:
# Palabras que MÁS se relacionan con...:
sg_2.wv.most_similar(positive=["rocinante"], topn=10)

[('cabestro', 0.4199143350124359),
 ('seguíale', 0.415421724319458),
 ('encaminó', 0.3921724855899811),
 ('picó', 0.38516107201576233),
 ('espuelas', 0.3826594650745392),
 ('embrazando', 0.3724251091480255),
 ('abrazar', 0.37177932262420654),
 ('sostenía', 0.3687582015991211),
 ('atravesado', 0.3685321509838104),
 ('reposado', 0.3625868260860443)]

Se puede observar que para el caso de "rocinante" las palabras más similares están estrechamente relacionadas a los caballos, tanto equipamiento como movimientos de los caballos.

In [18]:
# Palabras que MENOS se relacionan con...:
sg_2.wv.most_similar(negative=["quijote"], topn=10)

[('rico', 0.005084389355033636),
 ('sirva', 0.0009103221236728132),
 ('escoger', -0.011759641580283642),
 ('estimación', -0.013548364862799644),
 ('desengaño', -0.013570559211075306),
 ('poca', -0.020664053037762642),
 ('poetas', -0.021028881892561913),
 ('ocho', -0.022555021569132805),
 ('echar', -0.02315581776201725),
 ('felicísimo', -0.023315472528338432)]

Para el caso de palabras que menos se relacionan con "quijote" es interesante notar que son términos que realmente no tienen nada que ver con Don Quijote ya que a lo largo de la obra el mismo personaje hace referencia a su pobreza y desventuras.

#### Skip-Gram 2

In [20]:
# Palabras que MÁS se relacionan con...:
sg_5.wv.most_similar(positive=["quijote"], topn=10)

[('don', 0.648628294467926),
 ('jerónimo', 0.37555158138275146),
 ('sancho', 0.37298208475112915),
 ('hablas', 0.3480585813522339),
 ('dígote', 0.3472543954849243),
 ('tarfe', 0.3470093607902527),
 ('ta', 0.346200168132782),
 ('religioso', 0.3454519212245941),
 ('hiciéronlo', 0.33223816752433777),
 ('álvaro', 0.3312835991382599)]

Al aumentar la ventana de Skip-Gram a 5 se puede observar que se modifica ligeramente la lista de palabras más relacionadas a "quijote" ya que ahora por ejemplo aparece "sancho" y algunos otros términos que menciona el quijote.

In [21]:
# Palabras que MÁS se relacionan con...:
sg_5.wv.most_similar(positive=["sancho"], topn=10)

[('panza', 0.44954997301101685),
 ('quijote', 0.37298208475112915),
 ('dígote', 0.37013110518455505),
 ('estemos', 0.36586350202560425),
 ('hablas', 0.35954904556274414),
 ('majadero', 0.3571469783782959),
 ('hiciéronlo', 0.34590861201286316),
 ('que', 0.34542787075042725),
 ('jerónimo', 0.3406313359737396),
 ('decid', 0.336548388004303)]

Para el caso de "sancho" también se puede ver que se agregó la palabra "quijote", lo que es una señal de que este modelo capturó mejor la relación entre ambos personajes.

In [22]:
# Palabras que MÁS se relacionan con...:
sg_5.wv.most_similar(positive=["rocinante"], topn=10)

[('picó', 0.44040608406066895),
 ('cabestro', 0.40236878395080566),
 ('espuelas', 0.3921803832054138),
 ('estribo', 0.3911111056804657),
 ('embrazó', 0.37345385551452637),
 ('seguíale', 0.3642861843109131),
 ('rienda', 0.36416497826576233),
 ('embrazando', 0.3596392571926117),
 ('reposado', 0.357524037361145),
 ('descortés', 0.3509603440761566)]

En el caso de "rocinante" no hay tantas palabras nuevas pero sí cambió el orden de las palabras más similares.

In [23]:
# Palabras que MENOS se relacionan con...:
sg_5.wv.most_similar(negative=["quijote"], topn=10)

[('algodón', 0.046420346945524216),
 ('batallas', 0.011000814847648144),
 ('comedias', 0.0051581114530563354),
 ('escoger', 0.0016811976674944162),
 ('ciudades', -0.004375516902655363),
 ('volviese', -0.0045248232781887054),
 ('ingenio', -0.009265159256756306),
 ('disculpa', -0.009507724083960056),
 ('tenga', -0.01216969545930624),
 ('acordó', -0.012745779938995838)]

Es interesante notar que cambió la lista de palabras que menos se relacionan con "quijote" ya que ahora hay palabras nuevas que siguen teniendo sentido en este contexto como "comedias" o "ingenio".

#### CBOW 1

In [38]:
# Palabras que MÁS se relacionan con...:
cbow_2.wv.most_similar(positive=["quijote"], topn=10)

[('luis', 0.6069381833076477),
 ('lorenzo', 0.5668205618858337),
 ('fernando', 0.5638371706008911),
 ('álvaro', 0.5618709325790405),
 ('jerónimo', 0.5509666800498962),
 ('antonio', 0.5275198817253113),
 ('gaiferos', 0.4868934154510498),
 ('vicente', 0.4326227605342865),
 ('sancho', 0.4288356304168701),
 ('diego', 0.41858264803886414)]

Es interesante notar que con CBOW con una ventana de 2, las palabras que más se relacionan con "quijote" son en su mayoría nombres propios de otros personajes del libro y desapareció el término "don".

In [39]:
# Palabras que MÁS se relacionan con...:
cbow_2.wv.most_similar(positive=["sancho"], topn=10)

[('teresa', 0.6100258827209473),
 ('galeote', 0.5227469205856323),
 ('ambrosio', 0.4817162752151489),
 ('paje', 0.4511835277080536),
 ('quijote', 0.4288356304168701),
 ('sanchica', 0.4251628816127777),
 ('sansón', 0.42365336418151855),
 ('caminante', 0.42229875922203064),
 ('ricote', 0.4061884582042694),
 ('bosque', 0.40286457538604736)]

Es interesante notar que en el caso de "sancho" también aparecen más nombres propios relacionados al personaje.

In [40]:
# Palabras que MÁS se relacionan con...:
cbow_2.wv.most_similar(positive=["rocinante"], topn=10)

[('rucio', 0.4300481677055359),
 ('subir', 0.38905659317970276),
 ('cabestro', 0.38707876205444336),
 ('asno', 0.3805701732635498),
 ('abrazar', 0.37798911333084106),
 ('galope', 0.37211552262306213),
 ('encaminó', 0.36782729625701904),
 ('atravesado', 0.35878390073776245),
 ('caballo', 0.357482373714447),
 ('riendas', 0.3463549315929413)]

En el caso de "rocinante" pasa lo mismo que en los casos anteriores ya que la palabra con la que más se relaciona es "rucio", el asno de Sancho Panza. De hecho, la palabra "asno" también se relaciona con "rocinante", al igual que "caballo".

In [41]:
# Palabras que MENOS se relacionan con...:
cbow_2.wv.most_similar(negative=["quijote"], topn=10)

[('haciéndole', 0.27783945202827454),
 ('ciudades', 0.26492661237716675),
 ('despojos', 0.25490477681159973),
 ('mortales', 0.2476368099451065),
 ('creció', 0.23981042206287384),
 ('reprehensión', 0.23568738996982574),
 ('divinas', 0.23418255150318146),
 ('niños', 0.2289322018623352),
 ('tengamos', 0.2191506326198578),
 ('andanza', 0.21888260543346405)]

Las palabras que menos se relacionan con "quijote" cambia en comparación a los modelos anteriores ya que parecen haber desaparecido los adjetivos y ya no es tan simple determinar si tienen sentido o no.

#### CBOW 2

In [42]:
# Palabras que MÁS se relacionan con...:
cbow_5.wv.most_similar(positive=["quijote"], topn=10)

[('antonio', 0.5361063480377197),
 ('luis', 0.5349081158638),
 ('fernando', 0.5334556698799133),
 ('jerónimo', 0.5266017913818359),
 ('lorenzo', 0.5214809775352478),
 ('álvaro', 0.4668184220790863),
 ('gaiferos', 0.425245463848114),
 ('vicente', 0.4024867117404938),
 ('gregorio', 0.39953306317329407),
 ('diego', 0.39745011925697327)]

Al aumentar la ventana de CBOW a 5 se puede ver que siguen manteniendose como palabras más relacionadas a "quijote" los nombres propios de otros personajes. Es interesante notar que desaparece la palabra "sancho" de la lista.

In [43]:
# Palabras que MÁS se relacionan con...:
cbow_5.wv.most_similar(positive=["sancho"], topn=10)

[('teresa', 0.5848585963249207),
 ('galeote', 0.4552883207798004),
 ('sansón', 0.4220113158226013),
 ('licenciado', 0.41614583134651184),
 ('caminante', 0.38356268405914307),
 ('paje', 0.38196492195129395),
 ('ambrosio', 0.3710506558418274),
 ('bosque', 0.3673679530620575),
 ('labrador', 0.34853434562683105),
 ('ginés', 0.34641149640083313)]

Al igual que con "quijote", en el caso de "sancho" cambia ligeramente la lista de palabras, agregando algunos adjetivos del personaje.

In [44]:
# Palabras que MÁS se relacionan con...:
cbow_5.wv.most_similar(positive=["rocinante"], topn=10)

[('cabestro', 0.4502730667591095),
 ('subir', 0.4185652732849121),
 ('rucio', 0.4171329438686371),
 ('rienda', 0.40430504083633423),
 ('asiéndole', 0.402610719203949),
 ('estribo', 0.4014948010444641),
 ('abrazar', 0.3838617503643036),
 ('sacaron', 0.3793131709098816),
 ('encaminó', 0.37881869077682495),
 ('rocín', 0.37326234579086304)]

En el caso de "rocinante" es interesante notar que la lista de palabras obtenida es más similar a las que se obtuvo con Skip-Gram. Igualmente "rucio" sigue siendo una de las palabras más similares.

In [45]:
# Palabras que MENOS se relacionan con...:
cbow_5.wv.most_similar(negative=["quijote"], topn=10)

[('don', 0.2568643391132355),
 ('alcanzar', 0.2001873254776001),
 ('salí', 0.19099703431129456),
 ('so', 0.1905861496925354),
 ('felicísima', 0.18491026759147644),
 ('tengamos', 0.1843477487564087),
 ('aldonza', 0.18417906761169434),
 ('honesta', 0.18401111662387848),
 ('habedes', 0.1838729828596115),
 ('ofrecer', 0.18236349523067474)]

Es interesante notar que con CBOW con ventana de 5, "don" es una de las palabras que menos se relaciona con "quijote". Esto puede deberse a que el modelo capturó que la palabra "don" en realidad es un término relacionado a la nobleza y no está muy relacionado a "quijote".

### 5 - Visualizar agrupación de vectores

In [46]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

#### Skip-Gram 1

In [47]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(sg_2)

MAX_WORDS=250
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Comentarios:

- Se puede ver en la parte superior izquierda como "don" y "quijote" están prácticamente en la misma posición.
- Es interesante notar también en la parte inferior derecha que las palabras "caballeros" y "andantes" también están en posiciones muy cercanas.
- También se puede observar en la parte izquierda que "rocinante" está cerca a "caballo".
- En la parte central parecen estar agrupados los adverbios, posesivos y tipos de palabras similares.

#### Skip-Gram 2

In [48]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(sg_5)

MAX_WORDS=250
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Comentarios:

- Las palabras "caballeros" y "andantes" siguen en posiciones muy cercanas en la parte superior.
- "dulcinea" y "toboso" están prácticamente en la misma posición en la parte izquierda al igual que "vuesa" y "merced".
- Está todo bastante agrupado en el centro, pero igualmente se puede observar que "don" y "quijote" están bastante cerca.

#### CBOW 1

In [49]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(cbow_2)

MAX_WORDS=250
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Comentarios:

- Es interesante notar cómo agrupa muy bien los sinónimos, como por ejemplo en la parte derecha "dijo", "replicó" y "respondió".
- De igual manera se ven agrupadas todas las conjugaciones del verbo "haber" en la parte inferior derecha.
- Al igual que en Skip-Gram se puede ver que "caballeros" y "andantes" siguen muy cerca.
- Es interesante que "don" está cerca de "señor" y "amigo" pero también de "sancho".

#### CBOW 2

In [50]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(cbow_5)

MAX_WORDS=250
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Comentarios:

- Al igual que el CBOW anterior, parece agrupar muy bien los sinónimos y conjugaciones de los mismos verbos.
- Es interesante notar como "don" sigue cercano a "sancho" en lugar de a "quijote".

In [52]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/

import os

for name, model in models.items():
  os.mkdir(name)
  vectors = np.asarray(model.wv.vectors)
  labels = list(model.wv.index_to_key)

  np.savetxt(f"{name}/vectors.tsv", vectors, delimiter="\t")

  with open(f"{name}/labels.tsv", "w") as fp:
      for item in labels:
          fp.write("%s\n" % item)

### Conclusiones

- Con Skip-Gram, se observa que logra capturar las características distintivas de los diferentes personajes del libro, pero sin relacionarlos estrechamente entre sí.
- Por otro lado, el modelo CBOW tiende a agrupar mejor sinónimos, conjugaciones de verbos y algunos personajes, lo que sugiere que tiene una mayor capacidad para identificar relaciones semánticas.
- Vale la pena destacar que CBOW con ventana de 5 fue el único en determinar que "don" no era similar a "quijote" a pesar de aparecer juntos en casi todo el libro. Esto resalta la habilidad de CBOW para distinguir significados en función del contexto más amplio, en lugar de simplemente basarse en la proximidad de las palabras.
- En resumen, estos resultados sugieren que CBOW es más efectivo para agrupar palabras en función de su significado, mientras que Skip-Gram captura mejor las relaciones contextuales en las que aparecen las palabras.